# Read data

In [1]:
import pandas as pd
import os

In [2]:
path = "data/AIEP_common_user_questions_Kenyan_version.tsv"
basename = os.path.basename(path)
basename = os.path.splitext(basename)[0]

qa_df = pd.read_csv(path, sep="\t", header=0)
qa_df.describe(include="all")

,Question,Correct response,Follow-up questions needed,Comment
count,41,0.0,0.0,0.0
unique,41,NaN,NaN,NaN
top,What is the appropriate age for vaccination of...,NaN,NaN,NaN
freq,1,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN


In [3]:
qa_df = qa_df.rename(columns={"Question": "question"})

qa_df['question_id'] = qa_df.index + 1

In [4]:
qa_df

,question,Correct response,Follow-up questions needed,Comment,question_id
0,What is the appropriate age for vaccination of...,NaN,NaN,NaN,1
1,What should I do to prevent the mango tree fro...,NaN,NaN,NaN,2
2,What are alternative crops to cowdung?,NaN,NaN,NaN,3
3,What are the effects of marigold as a trap crop?,NaN,NaN,NaN,4
4,How do I prevent leaf blight in chili?,NaN,NaN,NaN,5
5,Can the quantity of grain be increased in the ...,NaN,NaN,NaN,6
6,How do I to recognize milk fever in cows?,NaN,NaN,NaN,7
7,Can animal feed be made at home?,NaN,NaN,NaN,8
8,How do I to choose a suitable breed of cow?,NaN,NaN,NaN,9
9,What are some regenerative farming strategies ...,NaN,NaN,NaN,10


# Set up API keys

In [5]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [6]:
def open_ended_prompt_simple(question, country):
    return (f"You provide smallholder farmers in {country} with agricultural advice and information. "
            f"Do not use more than 10 sentences.\n\nQuestion: {question}")

def open_ended_prompt_contextual(question, country):
    return (f"You provide smallholder farmers in {country} with high quality agricultural advice and information. "
            f"Please respond to the following question like an agronomist or agricultural extension agent would respond "
            f"to a smallholder farmer. Be factually correct, precise and make the response understandable for a smallholder farmer. "
            f"Do not use more than 10 sentences.\n\nQuestion: {question}")

def mcq_prompt(question, answer_options, country):
    return (f"You provide smallholder farmers in {country} with agricultural advice and information. "
            f"For the following multiple choice question, please select the correct answer(s) from options A to E and "
            f"provide a detailed rationale for your choice. There can be multiple correct answers.\n\n"
            f"Question: {question}\n\n"
            f"Answer options:\n{answer_options}\n\n"
            f"Your response must follow this format exactly:\n\n"
            f"Correct answer: [OPTION(S)]\n\n"
            f"Rationale: [Not longer than 10 sentences]")

In [7]:
import litellm

[2025-05-22 18:04:18 - httpx:1025 - INFO] HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


In [8]:
# set callbacks
litellm.success_callback = ["langfuse"]
litellm.failure_callback = ["langfuse"]

In [9]:
country = "Kenya"

In [10]:
def get_llm_answers(df, model, country, **kwargs):
    ids = []
    prompts = []
    answers = []

    for index, row in df.iterrows():
        question = row["question"]
        id = row["question_id"]
        prompt = open_ended_prompt_contextual(question, country)

        response = litellm.completion(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            **kwargs
        )

        # Extract the answer from the response
        answer = response["choices"][0]["message"]["content"]

        # Append the id, prompt, and answer to the lists
        ids.append(id)
        prompts.append(prompt)
        answers.append(answer)

    # Create a DataFrame from the lists
    results_df = pd.DataFrame({
        "question_id": ids,
        "prompt": prompts,
        "answer": answers
    })

    # Save the DataFrame to a CSV file
    model_name = model.replace("/", "-").replace(".", "-")
    csv_path = os.path.join("results", f"{basename}-{model_name}.csv")
    print(f"Saving results to {csv_path}")
    results_df.to_csv(csv_path, index=False)

    return results_df

# OpenAI

In [24]:
gpt_4o_df = get_llm_answers(qa_df, "openai/gpt-4o-2024-08-06", country)

Saving results to results/AIEP_common_user_questions_Kenyan_version-openai-gpt-4o-2024-08-06.csv


In [25]:
gpt_4o_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in Kenya with ...,"For vaccinating cows, it's important to follow..."
1,2,You provide smallholder farmers in Kenya with ...,To prevent your mango tree from rotting and fa...
2,3,You provide smallholder farmers in Kenya with ...,If you're looking for alternatives to cow dung...
3,4,You provide smallholder farmers in Kenya with ...,Marigold is an excellent trap crop that helps ...
4,5,You provide smallholder farmers in Kenya with ...,"To prevent leaf blight in chili, start by plan..."
5,6,You provide smallholder farmers in Kenya with ...,"Yes, it is possible to increase the quantity o..."
6,7,You provide smallholder farmers in Kenya with ...,"Milk fever, also known as hypocalcemia, often ..."
7,8,You provide smallholder farmers in Kenya with ...,"Yes, you can make animal feed at home, and it ..."
8,9,You provide smallholder farmers in Kenya with ...,Choosing the right breed of cow for your farm ...
9,10,You provide smallholder farmers in Kenya with ...,Regenerative farming focuses on improving soil...


In [26]:
o1_df = get_llm_answers(qa_df, "openai/o1-2024-12-17", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/AIEP_common_user_questions_Kenyan_version-openai-o1-2024-12-17.csv


In [27]:
o1_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in Kenya with ...,Different vaccines are given at different ages...
1,2,You provide smallholder farmers in Kenya with ...,"First, make sure water does not pool around th..."
2,3,You provide smallholder farmers in Kenya with ...,"Cow dung is animal manure, not a crop, so I’ll..."
3,4,You provide smallholder farmers in Kenya with ...,Marigolds release chemicals from their roots t...
4,5,You provide smallholder farmers in Kenya with ...,"1. Start with clean, certified seed or healthy..."
5,6,You provide smallholder farmers in Kenya with ...,"Yes, you can still increase grain yields witho..."
6,7,You provide smallholder farmers in Kenya with ...,Milk fever often happens shortly after a cow h...
7,8,You provide smallholder farmers in Kenya with ...,"Yes, you can make animal feed at home using lo..."
8,9,You provide smallholder farmers in Kenya with ...,"First, consider your farm’s environment, inclu..."
9,10,You provide smallholder farmers in Kenya with ...,"Try rotating your crops, especially with nutri..."


In [28]:
o3_df = get_llm_answers(qa_df, "openai/o3-2025-04-16", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/AIEP_common_user_questions_Kenyan_version-openai-o3-2025-04-16.csv


In [29]:
o3_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in Kenya with ...,Give a calf its first protection through colos...
1,2,You provide smallholder farmers in Kenya with ...,1. Plant or keep your mango in a spot that dra...
2,3,You provide smallholder farmers in Kenya with ...,"If you lack enough cow-dung, you can still fee..."
3,4,You provide smallholder farmers in Kenya with ...,1. When you plant a band or strip of marigold ...
4,5,You provide smallholder farmers in Kenya with ...,1. Start with clean seed or a tolerant variety...
5,6,You provide smallholder farmers in Kenya with ...,"Yes, you can still raise the grain yield even ..."
6,7,You provide smallholder farmers in Kenya with ...,Milk fever (hypocalcaemia) usually appears wit...
7,8,You provide smallholder farmers in Kenya with ...,"Yes, you can formulate good quality feed at ho..."
8,9,You provide smallholder farmers in Kenya with ...,1. First decide what you need most from the co...
9,10,You provide smallholder farmers in Kenya with ...,"1. Rotate your crops—e.g., maize this season, ..."


In [30]:
o4_mini_df = get_llm_answers(qa_df, "openai/o4-mini-2025-04-16", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/AIEP_common_user_questions_Kenyan_version-openai-o4-mini-2025-04-16.csv


In [31]:
o4_mini_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in Kenya with ...,1. A newborn calf gets protective antibodies f...
1,2,You provide smallholder farmers in Kenya with ...,"1. Mango trees hate waterlogged soil, so plant..."
2,3,You provide smallholder farmers in Kenya with ...,1. Cow dung is a good organic fertilizer but y...
3,4,You provide smallholder farmers in Kenya with ...,1. Marigolds (Tagetes spp.) used as a trap or ...
4,5,You provide smallholder farmers in Kenya with ...,"1. Use healthy, certified seed or a blight-res..."
5,6,You provide smallholder farmers in Kenya with ...,Yes. Grain yields depend on good soil fertilit...
6,7,You provide smallholder farmers in Kenya with ...,Milk fever usually shows up around calving whe...
7,8,You provide smallholder farmers in Kenya with ...,Yes. You can make balanced animal feed at home...
8,9,You provide smallholder farmers in Kenya with ...,1. First decide your main goal: milk productio...
9,10,You provide smallholder farmers in Kenya with ...,"1. Rotate your main crops (maize, beans, sorgh..."


# Anthropic

In [32]:
claude_3_5_df = get_llm_answers(qa_df, "anthropic/claude-3-5-sonnet-20241022", country)

Saving results to results/AIEP_common_user_questions_Kenyan_version-anthropic-claude-3-5-sonnet-20241022.csv


In [33]:
claude_3_5_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in Kenya with ...,Thank you for your question about cattle vacci...
1,2,You provide smallholder farmers in Kenya with ...,Thank you for your question about protecting y...
2,3,You provide smallholder farmers in Kenya with ...,I understand you're asking about alternatives ...
3,4,You provide smallholder farmers in Kenya with ...,Marigold is an excellent companion plant that ...
4,5,You provide smallholder farmers in Kenya with ...,"To prevent leaf blight in your chili plants, s..."
5,6,You provide smallholder farmers in Kenya with ...,"Yes, you can still achieve good grain yields e..."
6,7,You provide smallholder farmers in Kenya with ...,Milk fever usually occurs within 72 hours afte...
7,8,You provide smallholder farmers in Kenya with ...,"Yes, you can make quality animal feed at home ..."
8,9,You provide smallholder farmers in Kenya with ...,Thank you for your question about choosing a s...
9,10,You provide smallholder farmers in Kenya with ...,Let me share some practical regenerative farmi...


In [17]:
claude_3_7_df = get_llm_answers(qa_df, "anthropic/claude-3-7-sonnet-20250219", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/AIEP_common_user_questions_Kenyan_version-anthropic-claude-3-7-sonnet-20250219.csv


In [18]:
claude_3_7_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in Kenya with ...,# Appropriate Vaccination Ages for Cattle\n\nT...
1,2,You provide smallholder farmers in Kenya with ...,# Preventing Mango Tree Rot and Falling\n\nTo ...
2,3,You provide smallholder farmers in Kenya with ...,# Alternative Organic Fertilizers to Cow Manur...
3,4,You provide smallholder farmers in Kenya with ...,# Marigold as a Trap Crop\n\nMarigold plants w...
4,5,You provide smallholder farmers in Kenya with ...,# Preventing Leaf Blight in Chili Plants\n\nTo...
5,6,You provide smallholder farmers in Kenya with ...,# Increasing Grain Quantity Without Green Fodd...
6,7,You provide smallholder farmers in Kenya with ...,# Recognizing Milk Fever in Cows\n\nMilk fever...
7,8,You provide smallholder farmers in Kenya with ...,"# Making Animal Feed at Home\n\nYes, you can d..."
8,9,You provide smallholder farmers in Kenya with ...,# Choosing a Suitable Cow Breed\n\nTo choose t...
9,10,You provide smallholder farmers in Kenya with ...,# Regenerative Farming Strategies for Kenyan S...


Deepseek models don't support the `reasoning_effort` parameter.

In [11]:
deepseek_v3_df = get_llm_answers(qa_df, "deepseek/deepseek-chat", country, max_completion_tokens=25000)

Saving results to results/AIEP_common_user_questions_Kenyan_version-deepseek-deepseek-chat.csv


In [12]:
deepseek_v3_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in Kenya with ...,"For cows, the first vaccination should be give..."
1,2,You provide smallholder farmers in Kenya with ...,To prevent your mango tree from rotting and fa...
2,3,You provide smallholder farmers in Kenya with ...,If you need alternatives to cow dung for impro...
3,4,You provide smallholder farmers in Kenya with ...,Marigold is an excellent trap crop because it ...
4,5,You provide smallholder farmers in Kenya with ...,"To prevent leaf blight in chili, follow these ..."
5,6,You provide smallholder farmers in Kenya with ...,"Yes, you can increase grain yield even without..."
6,7,You provide smallholder farmers in Kenya with ...,Milk fever (hypocalcemia) occurs mostly in hig...
7,8,You provide smallholder farmers in Kenya with ...,"Yes, you can make high-quality animal feed at ..."
8,9,You provide smallholder farmers in Kenya with ...,"To choose a suitable cow breed, consider these..."
9,10,You provide smallholder farmers in Kenya with ...,Here are five simple regenerative farming stra...


In [13]:
deepseek_r1_df = get_llm_answers(qa_df, "deepseek/deepseek-reasoner", country, max_completion_tokens=25000)

Saving results to results/AIEP_common_user_questions_Kenyan_version-deepseek-deepseek-reasoner.csv


In [14]:
deepseek_r1_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in Kenya with ...,Calves should receive their first vaccinations...
1,2,You provide smallholder farmers in Kenya with ...,To prevent mango tree rot and collapse: \n1. ...
2,3,You provide smallholder farmers in Kenya with ...,Cow dung is not a crop but a fertilizer. If yo...
3,4,You provide smallholder farmers in Kenya with ...,Marigold acts as a trap crop by attracting pes...
4,5,You provide smallholder farmers in Kenya with ...,To prevent leaf blight in chili: \n1. Rotate ...
5,6,You provide smallholder farmers in Kenya with ...,"Yes, you can increase grain quantity without g..."
6,7,You provide smallholder farmers in Kenya with ...,"To recognize milk fever in cows, check these s..."
7,8,You provide smallholder farmers in Kenya with ...,"Yes, you can make high-quality animal feed at ..."
8,9,You provide smallholder farmers in Kenya with ...,"To choose a suitable cow breed, follow these s..."
9,10,You provide smallholder farmers in Kenya with ...,1. **Plant cover crops** like lablab or cowpea...


# Gemini

In [15]:
gemini_2_5_pro_df = get_llm_answers(qa_df, "gemini/gemini-2.5-pro-preview-05-06", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/AIEP_common_user_questions_Kenyan_version-gemini-gemini-2-5-pro-preview-05-06.csv


In [16]:
gemini_2_5_pro_df

,question_id,prompt,answer
0,1,You provide smallholder farmers in Kenya with ...,Good question! The right age to vaccinate your...
1,2,You provide smallholder farmers in Kenya with ...,"Okay, to help your mango tree stay strong and ..."
2,3,You provide smallholder farmers in Kenya with ...,"That's a good question! Actually, cowdung is n..."
3,4,You provide smallholder farmers in Kenya with ...,"Okay, that's a good question! Marigolds can be..."
4,5,You provide smallholder farmers in Kenya with ...,Good day! Leaf blight can be a real problem fo...
5,6,You provide smallholder farmers in Kenya with ...,"Yes, absolutely! You can definitely increase y..."
6,7,You provide smallholder farmers in Kenya with ...,"Alright, good question! Milk fever, or *kichaa..."
7,8,You provide smallholder farmers in Kenya with ...,"Yes, indeed, you can make animal feed at home ..."
8,9,You provide smallholder farmers in Kenya with ...,"Alright, choosing the right cow breed is a ver..."
9,10,You provide smallholder farmers in Kenya with ...,That's excellent you're looking into regenerat...


# Aggregate results

In [19]:
basename

'AIEP_common_user_questions_Kenyan_version'

In [20]:
import glob

# Get a list of all CSV files in the results directory
csv_files = glob.glob(f"results/{basename}-*.csv")
# Print the list of CSV files
print(csv_files)

['results/AIEP_common_user_questions_Kenyan_version-gemini-gemini-2-5-pro-preview-05-06.csv', 'results/AIEP_common_user_questions_Kenyan_version-openai-gpt-4o-2024-08-06.csv', 'results/AIEP_common_user_questions_Kenyan_version-anthropic-claude-3-7-sonnet-20250219.csv', 'results/AIEP_common_user_questions_Kenyan_version-openai-o3-2025-04-16.csv', 'results/AIEP_common_user_questions_Kenyan_version-openai-o1-2024-12-17.csv', 'results/AIEP_common_user_questions_Kenyan_version-openai-o4-mini-2025-04-16.csv', 'results/AIEP_common_user_questions_Kenyan_version-deepseek-deepseek-chat.csv', 'results/AIEP_common_user_questions_Kenyan_version-anthropic-claude-3-5-sonnet-20241022.csv', 'results/AIEP_common_user_questions_Kenyan_version-deepseek-deepseek-reasoner.csv']


In [21]:
len(csv_files)

9

In [22]:
# read all csv files, extract the model name from the filename, and concatenate them into a single dataframe
dfs = []
for file in csv_files:
    model_name = "-".join(os.path.basename(file).split("-")[1:]).split(".")[0]
    df = pd.read_csv(file)
    df["model"] = model_name
    dfs.append(df)

# concatenate all dataframes into a single dataframe
all_df = pd.concat(dfs, ignore_index=True)

# drop the prompt column
all_df = all_df.drop(columns=["prompt"])

# add country column
all_df["country"] = country

In [23]:
all_df

,question_id,answer,model,country
0,1,Good question! The right age to vaccinate your...,gemini-gemini-2-5-pro-preview-05-06,Kenya
1,2,"Okay, to help your mango tree stay strong and ...",gemini-gemini-2-5-pro-preview-05-06,Kenya
2,3,"That's a good question! Actually, cowdung is n...",gemini-gemini-2-5-pro-preview-05-06,Kenya
3,4,"Okay, that's a good question! Marigolds can be...",gemini-gemini-2-5-pro-preview-05-06,Kenya
4,5,Good day! Leaf blight can be a real problem fo...,gemini-gemini-2-5-pro-preview-05-06,Kenya
...,...,...,...,...
364,37,Paddy cultivation in Kenya is best timed with ...,deepseek-deepseek-reasoner,Kenya
365,38,Onions can be cultivated in Kenya during two m...,deepseek-deepseek-reasoner,Kenya
366,39,"To grow mushrooms, follow these steps: \n1. *...",deepseek-deepseek-reasoner,Kenya
367,40,Fungal diseases (often confused with edible mu...,deepseek-deepseek-reasoner,Kenya


In [24]:
# add the questions by merging with the original dataframe
all_df = all_df.merge(qa_df[["question_id", "question"]], on="question_id", how="left")

In [25]:
all_df

,question_id,answer,model,country,question
0,1,Good question! The right age to vaccinate your...,gemini-gemini-2-5-pro-preview-05-06,Kenya,What is the appropriate age for vaccination of...
1,2,"Okay, to help your mango tree stay strong and ...",gemini-gemini-2-5-pro-preview-05-06,Kenya,What should I do to prevent the mango tree fro...
2,3,"That's a good question! Actually, cowdung is n...",gemini-gemini-2-5-pro-preview-05-06,Kenya,What are alternative crops to cowdung?
3,4,"Okay, that's a good question! Marigolds can be...",gemini-gemini-2-5-pro-preview-05-06,Kenya,What are the effects of marigold as a trap crop?
4,5,Good day! Leaf blight can be a real problem fo...,gemini-gemini-2-5-pro-preview-05-06,Kenya,How do I prevent leaf blight in chili?
...,...,...,...,...,...
364,37,Paddy cultivation in Kenya is best timed with ...,deepseek-deepseek-reasoner,Kenya,When can you cultivate paddy in Kenya?
365,38,Onions can be cultivated in Kenya during two m...,deepseek-deepseek-reasoner,Kenya,When can you cultivate onions?
366,39,"To grow mushrooms, follow these steps: \n1. *...",deepseek-deepseek-reasoner,Kenya,How can I plant mushrooms?
367,40,Fungal diseases (often confused with edible mu...,deepseek-deepseek-reasoner,Kenya,What diseases are caused by mushrooms?


In [26]:
# add a response_id column
all_df["response_id"] = all_df.index + 1

In [27]:
all_df

,question_id,answer,model,country,question,response_id
0,1,Good question! The right age to vaccinate your...,gemini-gemini-2-5-pro-preview-05-06,Kenya,What is the appropriate age for vaccination of...,1
1,2,"Okay, to help your mango tree stay strong and ...",gemini-gemini-2-5-pro-preview-05-06,Kenya,What should I do to prevent the mango tree fro...,2
2,3,"That's a good question! Actually, cowdung is n...",gemini-gemini-2-5-pro-preview-05-06,Kenya,What are alternative crops to cowdung?,3
3,4,"Okay, that's a good question! Marigolds can be...",gemini-gemini-2-5-pro-preview-05-06,Kenya,What are the effects of marigold as a trap crop?,4
4,5,Good day! Leaf blight can be a real problem fo...,gemini-gemini-2-5-pro-preview-05-06,Kenya,How do I prevent leaf blight in chili?,5
...,...,...,...,...,...,...
364,37,Paddy cultivation in Kenya is best timed with ...,deepseek-deepseek-reasoner,Kenya,When can you cultivate paddy in Kenya?,365
365,38,Onions can be cultivated in Kenya during two m...,deepseek-deepseek-reasoner,Kenya,When can you cultivate onions?,366
366,39,"To grow mushrooms, follow these steps: \n1. *...",deepseek-deepseek-reasoner,Kenya,How can I plant mushrooms?,367
367,40,Fungal diseases (often confused with edible mu...,deepseek-deepseek-reasoner,Kenya,What diseases are caused by mushrooms?,368


In [28]:
# shuffle the dataframe
all_df = all_df.sample(frac=1, random_state=1337).reset_index(drop=True)
all_df

,question_id,answer,model,country,question,response_id
0,13,Thank you for your question about yellow pests...,anthropic-claude-3-5-sonnet-20241022,Kenya,How do I control yellow pests in paddy fields?,300
1,31,Let me explain how you can improve your soil h...,anthropic-claude-3-5-sonnet-20241022,Kenya,How can soil health be improved with crops?,318
2,17,"For best results, apply 10-15 tons of well-com...",deepseek-deepseek-reasoner,Kenya,How much cow dung should be applied per hectare?,345
3,25,1. Lay out simple drainage channels or ditches...,openai-o4-mini-2025-04-16,Kenya,What precautions should farmers take to preven...,230
4,29,Animals with internal parasites (such as worms...,openai-o1-2024-12-17,Kenya,What are the main symptoms of common parasitic...,193
...,...,...,...,...,...,...
364,4,Marigolds release chemicals from their roots t...,openai-o1-2024-12-17,Kenya,What are the effects of marigold as a trap crop?,168
365,28,Onion “dandruff” is not a fungus but a physiol...,openai-o4-mini-2025-04-16,Kenya,What causes dandruff in onions?,233
366,21,A mature cow (400-500kg) needs 10-15kg of dry ...,deepseek-deepseek-reasoner,Kenya,How much feed is needed for a rural cow?,349
367,26,Common diseases include East Coast fever (ECF)...,openai-o1-2024-12-17,Kenya,What are the common diseases that affect cattle?,190


In [29]:
# reorder the columns
all_df = all_df[["model", "question_id", "response_id", "question", "answer", "country"]]
all_df

,model,question_id,response_id,question,answer,country
0,anthropic-claude-3-5-sonnet-20241022,13,300,How do I control yellow pests in paddy fields?,Thank you for your question about yellow pests...,Kenya
1,anthropic-claude-3-5-sonnet-20241022,31,318,How can soil health be improved with crops?,Let me explain how you can improve your soil h...,Kenya
2,deepseek-deepseek-reasoner,17,345,How much cow dung should be applied per hectare?,"For best results, apply 10-15 tons of well-com...",Kenya
3,openai-o4-mini-2025-04-16,25,230,What precautions should farmers take to preven...,1. Lay out simple drainage channels or ditches...,Kenya
4,openai-o1-2024-12-17,29,193,What are the main symptoms of common parasitic...,Animals with internal parasites (such as worms...,Kenya
...,...,...,...,...,...,...
364,openai-o1-2024-12-17,4,168,What are the effects of marigold as a trap crop?,Marigolds release chemicals from their roots t...,Kenya
365,openai-o4-mini-2025-04-16,28,233,What causes dandruff in onions?,Onion “dandruff” is not a fungus but a physiol...,Kenya
366,deepseek-deepseek-reasoner,21,349,How much feed is needed for a rural cow?,A mature cow (400-500kg) needs 10-15kg of dry ...,Kenya
367,openai-o1-2024-12-17,26,190,What are the common diseases that affect cattle?,Common diseases include East Coast fever (ECF)...,Kenya


In [30]:
# save the dataframe to a CSV file
all_df.to_csv(f"{basename}-all.csv", index=False)